# 特征工程主要将明显提升模型的效果与训练时间，通常包括以下内容：

## 1.缺失值处理；（部分模型可自动填充）

## 2.异常值剔除；（通常需根据结果反馈与业务理解进行多次尝试迭代）

## 3.数据格式转化；（分类变量转化，时间格式转化，连续变量离散化等）

## 4.特征衍生；（交叉组合/派生/降维等等）

## 5.特征筛选（过滤法/包装法/嵌入法）

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import re
import datetime

In [2]:
def features_dict(df,columns):   #离散变量转化映射表，df为数据表，columns为目标列
    features_dict = {}
    i = 1
    list_mid = list(set(df[columns].to_list()))
    list_mid.sort()
    for each in list_mid:
        features_dict[each] = i
        i = i+1
    return features_dict

def features_trans(columns_values,features_dict):  #离散特征转化，columns_values为目标列，features_dict为特征映射表
    return features_dict[columns_values]

In [3]:
#数据读取
df_train = pd.read_csv(r"C:\Users\17738\贷款违约率预测\数据集\train.csv")
df_test = pd.read_csv(r"C:\Users\17738\贷款违约率预测\数据集\testA.csv")

## 缺失值处理

### 缺失值这里的整体思路如下：
    1.常规情况下，宁缺毋滥，如果缺失值数据量较少，相比于探索其缺失规律，直接剔除是一个更直接快速的方法；
        （1）确认缺失数据的比例；
        （2）确认缺失数据是否随机分布；
        （3）当缺失数据符合以上两点条件时，可以优先考虑剔除缺失值；
    2.当缺失值过多或缺失数据的比例分布与整体不一致时，直接的剔除可能会影响模型的泛化能力，优先考虑填充缺失值（统计方式填充/模型预测）

In [4]:
# 缺失值比例判断
df_train.isnull().sum().sort_values()

id                        0
totalAcc                  0
earliesCreditLine         0
revolBal                  0
pubRec                    0
openAcc                   0
ficoRangeHigh             0
ficoRangeLow              0
delinquency_2years        0
regionCode                0
initialListStatus         0
isDefault                 0
purpose                   0
verificationStatus        0
annualIncome              0
homeOwnership             0
policyCode                0
subGrade                  0
grade                     0
installment               0
interestRate              0
term                      0
loanAmnt                  0
issueDate                 0
applicationType           0
postCode                  1
employmentTitle           1
title                     1
dti                     239
pubRecBankruptcies      405
revolUtil               531
n10                   33239
n4                    33239
n13                   40270
n12                   40270
n9                  

In [5]:
#剔除全部列缺失值后的数据大小
df_train.dropna().shape[0]

686195

In [6]:
#缺失数据随机分布（缺失数据比例是否与训练集标签比例类似，在数据探索阶段以计算正负样本比例为1：4）
df_miss = pd.DataFrame(df_train.isnull().sum()).copy()
df_miss.rename(columns = {0:"缺失值"},inplace = True)
df_miss_columns = df_miss[df_miss["缺失值"]>0].index.to_list()  #少量的缺失值比例不具备参考性

for each in df_miss_columns:
    df_mid = df_train[df_train[each].isnull() == True].copy()
    miss_all_count = df_mid.shape[0]
    miss_y0_count = df_mid[df_mid["isDefault"].isnull() == 0].shape[0]
    miss_y1_count = df_mid[df_mid["isDefault"].isnull() == 1].shape[0]
    
    if miss_y1_count==0:
        print("缺失值正负比例探索",each,miss_y0_count,miss_y1_count)
    else:
        print("缺失值正负比例探索",each,miss_y0_count/miss_y1_count,miss_y0_count,miss_y1_count)

缺失值正负比例探索 employmentTitle 1 0
缺失值正负比例探索 employmentLength 46799 0
缺失值正负比例探索 postCode 1 0
缺失值正负比例探索 dti 239 0
缺失值正负比例探索 pubRecBankruptcies 405 0
缺失值正负比例探索 revolUtil 531 0
缺失值正负比例探索 title 1 0
缺失值正负比例探索 n0 40270 0
缺失值正负比例探索 n1 40270 0
缺失值正负比例探索 n2 40270 0
缺失值正负比例探索 n3 40270 0
缺失值正负比例探索 n4 33239 0
缺失值正负比例探索 n5 40270 0
缺失值正负比例探索 n6 40270 0
缺失值正负比例探索 n7 40270 0
缺失值正负比例探索 n8 40271 0
缺失值正负比例探索 n9 40270 0
缺失值正负比例探索 n10 33239 0
缺失值正负比例探索 n11 69752 0
缺失值正负比例探索 n12 40270 0
缺失值正负比例探索 n13 40270 0
缺失值正负比例探索 n14 40270 0


从输出结果来看，缺失值都为正样本，依据业务经验初步定位此情况是具有某种明显规律的缺失（比如特地某类特定用户的某项特征无法获取），下面进行两方面的探索：

1.缺失数据是否多列是同时出现的(用于评估缺失值填充影响的数据量，用于推测缺失数据的缺失特征)

2.测试集缺失数据的比例是否与训练集相近；（佐证）

In [7]:
#判断缺失值是否同增通减
df_miss_columns_fig = df_miss_columns.copy()
df_miss_columns_fig.append("features")
df_mid2 = pd.DataFrame(columns = [df_miss_columns_fig])
    
for each in df_miss_columns:
    df_mid = df_train[df_train[each].isnull() == True][df_miss_columns].copy()
    
    mid = df_mid.isnull().sum().to_list()
    mid.append(df_mid.shape[0])
    
    df_mid2.loc[df_mid2.shape[0]] = mid
df_mid2

,employmentTitle,employmentLength,postCode,dti,pubRecBankruptcies,revolUtil,title,n0,n1,n2,...,n6,n7,n8,n9,n10,n11,n12,n13,n14,features
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,46799,0,227,0,32,0,1209,1209,1209,...,1209,1209,1209,1209,965,3206,1209,1209,1209,46799
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,227,0,239,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,239
4,0,0,0,0,405,0,0,405,405,405,...,405,405,405,405,405,405,405,405,405,405
5,0,32,0,1,0,531,0,58,58,58,...,58,58,58,58,54,58,58,58,58,531
6,0,0,0,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
7,0,1209,0,0,405,58,1,40270,40270,40270,...,40270,40270,40270,40270,33239,40270,40270,40270,40270,40270
8,0,1209,0,0,405,58,1,40270,40270,40270,...,40270,40270,40270,40270,33239,40270,40270,40270,40270,40270
9,0,1209,0,0,405,58,1,40270,40270,40270,...,40270,40270,40270,40270,33239,40270,40270,40270,40270,40270


从数据结果来看，缺失值这里确实同增同减相互关联

In [8]:
df_miss_train = pd.DataFrame(df_train[df_miss_columns].isnull().sum()/8000)
df_miss_test = pd.DataFrame(df_test[df_miss_columns].isnull().sum()/2000)   #计算百分比，所以少两位数
df_miss_merge = pd.merge(df_miss_train,df_miss_test,left_index = True, right_index=True,how = "outer")
df_miss_merge

,0_x,0_y
employmentTitle,0.000125,0.0000
employmentLength,5.849875,5.8710
postCode,0.000125,0.0000
dti,0.029875,0.0305
pubRecBankruptcies,0.050625,0.0580
revolUtil,0.066375,0.0635
title,0.000125,0.0000
n0,5.033750,5.0555
n1,5.033750,5.0555
n2,5.033750,5.0555


从数据结果来看，缺失值的分布在测试集与训练集是一致的，因此可以初步做出假设“是具有某种明显规律的常态化缺失（比如特地某类特定用户的某项特征无法获取）” ；我们将缺失值初步填充为一个独立的点，此处设定为"-9999"。（填充原因：1.为了让模型可以更好的表达缺失即正样本这一对应关系；2.相比于均值等填充方式，该方式大概率不会引入噪音；）

此处确定了缺失值的填充策略，但为了不影响后续的数据处理，此处先不进行填充

对于异常值，一般在模型结果出来之后结果模型结果与业务经验对其进行修正更客观些，此处暂不做特殊处理;

## 数据格式转化

1.将非数值型数据转化为模型可以识别的数据类型；

2.将分类变量转化为哑变量或onehot形式；

3.将连续型变量进行分箱操作，提高模型的稳定性（通常需根据结果反馈与业务理解进行多次尝试迭代）；

In [9]:
#将在数据探索阶段发现的特殊字段进行格式转化
# grade 
# subGrade 
# employmentLength 
# issueDate
# earliesCreditLine 

先观察下字段具体取值

In [10]:
df_train.groupby(["grade"]).count().head(2)

,id,loanAmnt,term,interestRate,installment,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
grade,,,,,,,,,,,,,,,,,,,,,
A,139661,139661,139661,139661,139661,139661,139661,132093,139661,139661,...,129741,129741,129741,129741,129741,131026,127045,129741,129741,129741
B,233690,233690,233690,233690,233690,233690,233690,220152,233690,233690,...,220855,220855,220855,220855,220855,223298,212757,220855,220855,220855


In [11]:
df_train.groupby(["subGrade"]).count().head(2)

,id,loanAmnt,term,interestRate,installment,grade,employmentTitle,employmentLength,homeOwnership,annualIncome,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
subGrade,,,,,,,,,,,,,,,,,,,,,
A1,25909,25909,25909,25909,25909,25909,25909,24403,25909,25909,...,24604,24604,24604,24604,24604,24816,24202,24604,24604,24604
A2,22124,22124,22124,22124,22124,22124,22124,20925,22124,22124,...,20586,20586,20586,20586,20586,20776,20178,20586,20586,20586


In [12]:
df_train.groupby(["earliesCreditLine"]).count().head(2)

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
earliesCreditLine,,,,,,,,,,,,,,,,,,,,,
Apr-1955,2,2,2,2,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2
Apr-1958,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [13]:
df_train.groupby(["issueDate"]).count().head(2)

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
issueDate,,,,,,,,,,,,,,,,,,,,,
2007-06-01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2007-07-01,21,21,21,21,21,21,21,21,21,21,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_train.groupby(["earliesCreditLine"]).count().head(2)

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,...,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
earliesCreditLine,,,,,,,,,,,,,,,,,,,,,
Apr-1955,2,2,2,2,2,2,2,2,1,2,...,2,2,2,2,2,2,2,2,2,2
Apr-1958,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


有明显等级特征的特征，我们将其初步转化为递增序列

In [15]:
#生成特征映射表
grade_dict = features_dict(df_train,"grade")
subGrade_dict = features_dict(df_train,"subGrade")

#进行特征转化
df_train["grade"] = df_train.apply(lambda x : features_trans(x["grade"],grade_dict),axis = 1 )
df_train["subGrade"] = df_train.apply(lambda x : features_trans(x["subGrade"],subGrade_dict),axis = 1 )

对于时间类特征，我们选择一个基准值，然后用特征当前时间减去基准值得到的数据作为喂入模型的特征

In [16]:
#将字符串格式转化为时间格式
df_train['earliesCreditLine'] = pd.to_datetime(df_train['earliesCreditLine'])
df_train['issueDate'] = pd.to_datetime(df_train['issueDate'])

#以1900作为基数进行计算
df_train["earliesCreditLine"] = df_train["earliesCreditLine"].apply(lambda x : int(re.search(("\d+"),str(x-datetime.datetime(1900,1,1))).group(0)))
df_train["issueDate"] = df_train["issueDate"].apply(lambda x : int(re.search(("\d+"),str(x-datetime.datetime(1900,1,1))).group(0)))

此处要对employmentLength字段使用正则提取年份数据

In [17]:
df_train["employmentLength"] = df_train["employmentLength"].fillna("-9999")  #为了正则的顺利使用，此处按照缺失值的填充策先行进行字符串填充
df_train["employmentLength"] = df_train["employmentLength"].apply(lambda x : re.search("(.*?) ",str(x)).group(0) if x != "-9999" else x)
df_train["employmentLength"] = df_train["employmentLength"].apply(lambda x : 10 if x=="10+ " else 0.5 if x=="< " else x)
df_train["employmentLength"] = df_train["employmentLength"].astype(float)

缺失值填充

In [18]:
df_train = df_train.fillna(-9999)

In [19]:
#特征字段格式检查
for each in df_train.columns:
    if str(df_train[each].dtypes)!= "float64" and str(df_train[each].dtypes)!= "int64":
        print(each,str(df_train[each].dtypes))

异常值剔除/特征筛选将在模型训练之后进行探索剔除；

## 特征衍生

在模型训练之前，我们可以衍生一批具有信息量与区分度的特征集合，以便模型在训练时更大的提升空间。此处我们要进行的具体工作内容为：

1.基于业务意义的经验衍生（收入信息，贷款信息，行为特征信息）

    收入相关：
    
        累计总收入
        
        年均收入
        
    贷款相关：
    
        可用信用额度（总体）
        
        fico均值
        
    行为特征相关：
    
        贬损公共记录未清除数量；
        
        所在地区贷款人数；
        
        
2.基于统计的特征验证：


    1.对n系列特征的总和，均值，最大值，最小值；
    
    2.对n系列特征的多列求和；
    
    3.n系列特征分位数； 

In [20]:
#累计总收入
df_train["total_Income"] = df_train['annualIncome'] * df_train['employmentLength']
#年均收入
df_train["avg_Income"] = df_train['annualIncome'] / df_train['employmentLength']

#可用信用额度（总体）
df_train["avg_Income"] = df_train['openAcc'] + df_train['totalAcc']
#fico均值
df_train["avg_Income"] = df_train['ficoRangeLow'] + df_train['ficoRangeHigh']
#贬损公共记录未清除数量
df_train["avg_Income"] = df_train['pubRec'] - df_train['pubRecBankruptcies']

#所在地区贷款人数，所在地区贷款逾期人数；
df_region_count = df_train.groupby("regionCode").count().reset_index(drop=False)[["regionCode","isDefault"]].copy()
df_region_count.rename(columns = {"isDefault":"region_count"},inplace = True)

df_train = pd.merge(df_train,df_region_count,on='regionCode',how = "left")

In [21]:
#构建n系列列名列表
n_columns = []
for i in range(0,15):
    mid = "n" + str(i)
    n_columns.append(mid)

In [22]:

for each in n_columns:
    df_mid = df_train[[each]][df_train[each]>-9998.0].copy()
    
    #n系列总和 均值 最大值 最小值 
    df_train[each+"_sum"] = float(df_mid.sum())
    df_train[each+"_mean"] = float(df_mid.mean())
    df_train[each+"_max"] = float(df_mid.max())
    df_train[each+"_min"] = float(df_mid.min())
    
    #特征分位数
    df_train[each+"_0.25"] = df_mid[each].quantile(0.25)
    df_train[each+"_0.5"] = df_mid[each].quantile(0.5)
    df_train[each+"_0.75"] = df_mid[each].quantile(0.75)

#n系列多列总和
df_train["n_sum"] = df_train[n_columns].sum(axis=1)

In [23]:
#保存数据
df_train.to_csv(r"C:\Users\17738\贷款违约率预测\数据集\train_clean.csv",index = None)